In [1]:
import numpy                as np
import matplotlib.pyplot    as plt
import bsplines             as bsp

import STRUPHY_pusher       as pic_pusher
import STRUPHY_fields       as pic_fields
import STRUPHY_accumulation as pic_accumu

In [7]:
Nel = [3, 4, 6]            # mesh generation on logical domain
bc  = [True, True, True]   # boundary conditions
p   = [2, 2, 3]            # splines degrees

L   = [2., 3., 1.]         # box lengthes of physical domain
Np  = int(100)             # number of particles

el_b     = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                           # element boundaries
T        = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]       # knot vectors
t        = [T[1:-1] for T in T]                                                    # reduced knot vectors
Nbase0   = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                      # number of basis functions in V0
delta    = [1/Nel for Nel in Nel]

dt       = 0.15

particles       = np.empty((Np, 7), dtype=float, order='F')
particles[:, :] = np.random.rand(Np, 7)

spans0       = np.empty((Np, 3), dtype=int, order='F')
spans0[:, 0] = np.floor(particles[:, 0]*Nel[0]).astype(int) + p[0]
spans0[:, 1] = np.floor(particles[:, 1]*Nel[1]).astype(int) + p[1]
spans0[:, 2] = np.floor(particles[:, 2]*Nel[2]).astype(int) + p[2]

b1 = np.zeros((Nel[0], Nel[1], Nel[2]), order='F')
b2 = np.zeros((Nel[0], Nel[1], Nel[2]), order='F')
b3 = np.zeros((Nel[0], Nel[1], Nel[2]), order='F')

u1 = np.zeros((Nel[0], Nel[1], Nel[2]), order='F')
u2 = np.zeros((Nel[0], Nel[1], Nel[2]), order='F')
u3 = np.zeros((Nel[0], Nel[1], Nel[2]), order='F')

b1[:, :, :] = np.random.rand(Nel[0], Nel[1], Nel[2]) - 0.5
b2[:, :, :] = np.random.rand(Nel[0], Nel[1], Nel[2]) - 0.5
b3[:, :, :] = np.random.rand(Nel[0], Nel[1], Nel[2]) - 0.5

u1[:, :, :] = np.random.rand(Nel[0], Nel[1], Nel[2]) - 0.5
u2[:, :, :] = np.random.rand(Nel[0], Nel[1], Nel[2]) - 0.5
u3[:, :, :] = np.random.rand(Nel[0], Nel[1], Nel[2]) - 0.5

DF     = np.array([[  L[0], 0., 0.], [0.,   L[1], 0.], [0., 0.,   L[2]]])
DFinv  = np.array([[1/L[0], 0., 0.], [0., 1/L[1], 0.], [0., 0., 1/L[2]]])

G      = np.array([[  L[0]**2, 0., 0.], [0.,   L[1]**2, 0.], [0., 0.,   L[2]**2]])
Ginv   = np.array([[1/L[0]**2, 0., 0.], [0., 1/L[1]**2, 0.], [0., 0., 1/L[2]**2]])

g_sqrt = L[0]*L[1]*L[2]

Beq    = g_sqrt*DFinv.dot(np.array([0., 0., 1.]))
Ueq    = DF.T.dot(np.array([0.02, 0., 0.]))

# ================ coefficients for pp-forms in interval [0, delta] (N and D) ==================
pp0 = []
pp1 = []

for i in range(3):
    if p[i] == 3:
        pp0.append(np.asfortranarray([[1/6, -1/(2*delta[i]), 1/(2*delta[i]**2), -1/(6*delta[i]**3)], [2/3, 0., -1/delta[i]**2, 1/(2*delta[i]**3)], [1/6, 1/(2*delta[i]), 1/(2*delta[i]**2), -1/(2*delta[i]**3)], [0., 0., 0., 1/(6*delta[i]**3)]]))
        pp1.append(np.asfortranarray([[1/2, -1/delta[i], 1/(2*delta[i]**2)], [1/2, 1/delta[i], -1/delta[i]**2], [0., 0., 1/(2*delta[i]**2)]])/delta[i])
    elif p[i] == 2:
        pp0.append(np.asfortranarray([[1/2, -1/delta[i], 1/(2*delta[i]**2)], [1/2, 1/delta[i], -1/delta[i]**2], [0., 0., 1/(2*delta[i]**2)]]))
        pp1.append(np.asfortranarray([[1., -1/delta[i]], [0., 1/delta[i]]])/delta[i])
    else:
        print('So far only cubic and quadratic splines implemented!')
# ==============================================================================================

np.save('test_particles', particles)

np.save('b1_coeff', b1)
np.save('b2_coeff', b2)
np.save('b3_coeff', b3)


np.save('u1_coeff', u1)
np.save('u2_coeff', u2)
np.save('u3_coeff', u3)

In [8]:
particles = np.empty((Np, 7), dtype=float, order='F')
spans0    = np.empty((Np, 3), dtype=int, order='F')

b1 = np.empty((Nel[0], Nel[1], Nel[2]), order='F')
b2 = np.empty((Nel[0], Nel[1], Nel[2]), order='F')
b3 = np.empty((Nel[0], Nel[1], Nel[2]), order='F')

u1 = np.empty((Nel[0], Nel[1], Nel[2]), order='F')
u2 = np.empty((Nel[0], Nel[1], Nel[2]), order='F')
u3 = np.empty((Nel[0], Nel[1], Nel[2]), order='F')

U_part = np.empty((Np, 3), dtype=float, order='F')
B_part = np.empty((Np, 3), dtype=float, order='F')


particles[:, :] = np.load('test_particles.npy')

b1[:, :, :] = np.load('b1_coeff.npy')
b2[:, :, :] = np.load('b2_coeff.npy')
b3[:, :, :] = np.load('b3_coeff.npy')

u1[:, :, :] = np.load('u1_coeff.npy')
u2[:, :, :] = np.load('u2_coeff.npy')
u3[:, :, :] = np.load('u3_coeff.npy')


spans0[:, 0] = np.floor(particles[:, 0]*Nel[0]).astype(int) + p[0]
spans0[:, 1] = np.floor(particles[:, 1]*Nel[1]).astype(int) + p[1]
spans0[:, 2] = np.floor(particles[:, 2]*Nel[2]).astype(int) + p[2]

In [6]:
# test evaluate_1form
pic_fields.evaluate_1form(particles[:, 0:3], p, spans0, Nbase0, Np, u1, u2, u3, Ueq, pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], U_part)

U_part_parallel = np.load('U_part_parallel.npy')
print(np.allclose(U_part_parallel, U_part))

True


In [7]:
# test evaluate_2form
pic_fields.evaluate_2form(particles[:, 0:3], p, spans0, Nbase0, Np, b1, b2, b3, Beq, pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], B_part)

B_part_parallel = np.load('B_part_parallel.npy')
print(np.allclose(B_part_parallel, B_part))

True


In [6]:
# test pusher_step3
pic_fields.evaluate_1form(particles[:, 0:3], p, spans0, Nbase0, Np, u1, u2, u3, Ueq, pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], U_part)
pic_fields.evaluate_2form(particles[:, 0:3], p, spans0, Nbase0, Np, b1, b2, b3, Beq, pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], B_part)
pic_pusher.pusher_step3(particles, L, dt, B_part, U_part)
particles_parallel = np.load('particles_after_push3.npy')
print(np.allclose(particles_parallel, particles))

True


In [9]:
# test pusher_step4    
pic_pusher.pusher_step4(particles, L, dt)
particles_parallel = np.load('particles_after_push4.npy')
print(np.allclose(particles_parallel, particles))

True


In [5]:
# test pusher_step5   
pic_fields.evaluate_2form(particles[:, 0:3], p, spans0, Nbase0, Np, b1, b2, b3, Beq, pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], B_part)
pic_pusher.pusher_step5(particles, L, dt, B_part)
particles_parallel = np.load('particles_after_push5.npy')
print(np.allclose(particles_parallel, particles))

True


In [4]:
# test accumulation_step1   
mat12  = np.empty((Nbase0[0], Nbase0[1], Nbase0[2], Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')
mat13  = np.empty((Nbase0[0], Nbase0[1], Nbase0[2], Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')
mat23  = np.empty((Nbase0[0], Nbase0[1], Nbase0[2], Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')

pic_fields.evaluate_2form(particles[:, 0:3], p, spans0, Nbase0, Np, b1, b2, b3, Beq, pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], B_part)
pic_accumu.accumulation_step1(particles, p, spans0, Nbase0, T[0], T[1], T[2], t[0], t[1], t[2], L, B_part, mat12, mat13, mat23)
mat12_parallel = np.load('mat12_step1.npy')
mat13_parallel = np.load('mat13_step1.npy')
mat23_parallel = np.load('mat23_step1.npy')
print(np.allclose(mat12_parallel, mat12))
print(np.allclose(mat13_parallel, mat13))
print(np.allclose(mat23_parallel, mat23))

True
True
True


In [10]:
# test accumulation_step3   
mat11  = np.empty((Nbase0[0], Nbase0[1], Nbase0[2], Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')
mat12  = np.empty((Nbase0[0], Nbase0[1], Nbase0[2], Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')
mat13  = np.empty((Nbase0[0], Nbase0[1], Nbase0[2], Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')
mat22  = np.empty((Nbase0[0], Nbase0[1], Nbase0[2], Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')
mat23  = np.empty((Nbase0[0], Nbase0[1], Nbase0[2], Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')
mat33  = np.empty((Nbase0[0], Nbase0[1], Nbase0[2], Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')

vec1   = np.empty((Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')
vec2   = np.empty((Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')
vec3   = np.empty((Nbase0[0], Nbase0[1], Nbase0[2]), dtype=float, order='F')


pic_fields.evaluate_2form(particles[:, 0:3], p, spans0, Nbase0, Np, b1, b2, b3, Beq, pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], B_part)
pic_accumu.accumulation_step3(particles, p, spans0, Nbase0, T[0], T[1], T[2], t[0], t[1], t[2], L, B_part, mat11, mat12, mat13, mat22, mat23, mat33, vec1, vec2, vec3)

mat11_parallel = np.load('mat11_step3.npy')
mat12_parallel = np.load('mat12_step3.npy')
mat13_parallel = np.load('mat13_step3.npy')

mat22_parallel = np.load('mat22_step3.npy')
mat23_parallel = np.load('mat23_step3.npy')
mat33_parallel = np.load('mat33_step3.npy')

vec1_parallel = np.load('vec1_step3.npy')
vec2_parallel = np.load('vec2_step3.npy')
vec3_parallel = np.load('vec3_step3.npy')


print(np.allclose(mat11_parallel, mat11))
print(np.allclose(mat12_parallel, mat12))
print(np.allclose(mat13_parallel, mat13))
print(np.allclose(mat22_parallel, mat22))
print(np.allclose(mat23_parallel, mat23))
print(np.allclose(mat33_parallel, mat33))
print(np.allclose(vec1_parallel, vec1))
print(np.allclose(vec2_parallel, vec2))
print(np.allclose(vec3_parallel, vec3))

True
True
True
True
True
True
True
True
True
